In [1]:
import pandas as pd
import numpy as np

import re
import matplotlib.pyplot as plt
import plotly.express as px

import collections
from plotly import graph_objects as go


import sys
sys.path.append("..")

flatten = lambda t: [item for sublist in t for item in sublist]
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [2]:
comment_df = pd.read_excel(r"..\data\psaw_comments_from_sept1_pol.xlsx")
comment_df["lens"] = comment_df.text.str.split().map(lambda x: len(x))
comment_df.head()

,Unnamed: 0,comment_id,text,author,upvotes,timestamp,post_id,lens
0,0,hfu3t3z,Minden oké otthon?,drakvuf,15,1633686153,q3uef4,3
1,1,hfu3wlv,"Ez azért elég izzadtságszagú azt tekintve, hog...",Debre1024,6,1633686239,q3uef4,15
2,2,hfu491h,téged ki bántott :(,Endymion2748,6,1633686552,q3uef4,4
3,3,hfu407d,Még nincs szombat...,LazyTomTom,3,1633686328,q3uef4,3
4,4,hfu4nyg,"Ezazz! Te is szereted ha megbasznak szárazon, ...",DrTorrente,3,1633686923,q3uef4,21


In [3]:
comment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12950 entries, 0 to 12949
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  12950 non-null  int64 
 1   comment_id  12950 non-null  object
 2   text        12950 non-null  object
 3   author      12950 non-null  object
 4   upvotes     12950 non-null  int64 
 5   timestamp   12950 non-null  int64 
 6   post_id     12950 non-null  object
 7   lens        12950 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 809.5+ KB


In [4]:
w_http = comment_df["text"].str.contains("http")
#w_http[w_http]
w_http[w_http]

7        True
18       True
25       True
31       True
52       True
         ... 
12918    True
12925    True
12933    True
12944    True
12947    True
Name: text, Length: 589, dtype: bool

In [5]:
print("Before:\n",comment_df["text"].iloc[25])
print("Num of http:", comment_df["text"].map(lambda x: "http" in x).sum())


comment_df["text"] = comment_df["text"].map( lambda x: re.sub(r'http\S+', '', x) )
print("\nAfter:\n",comment_df["text"].iloc[25])
print("Num of http:",comment_df.text.map(lambda x: "http" in x).sum())
print("Num of https:",comment_df.text.map(lambda x: "https" in x).sum())
#Remove web links


Before:
 [u/shiffer_andras](https://youtu.be/dQw4w9WgXcQ) vélemény?
Num of http: 589

After:
 [u/shiffer_andras]( vélemény?
Num of http: 0
Num of https: 0


In [6]:
#remove user mentions 

print("Before:\n",comment_df["text"].iloc[25])
print("Num of user mentions:", comment_df["text"].map(lambda x: "u/" in x).sum())

comment_df["text"] = comment_df["text"].map( lambda x: re.sub(r"u/\S+", "", x))

print("\nAfter:\n",comment_df["text"].iloc[25])
print("Num of user mentions:",comment_df.text.map(lambda x: "u/" in x).sum())
print("Num of user mentions:",comment_df.text.map(lambda x: "u/" in x).sum())

Before:
 [u/shiffer_andras]( vélemény?
Num of user mentions: 47

After:
 [ vélemény?
Num of user mentions: 0
Num of user mentions: 0


In [7]:
#remove emojis
comment_df["text"] = comment_df["text"].map(deEmojify)

In [46]:
from string import punctuation, digits




#Remove numbers
for d in digits:
    comment_df["text"] = comment_df["text"].str.replace(d, "")

#Remove punctuations
for s in punctuation:
    comment_df["text"] = comment_df["text"].str.replace(s, " ")

#Make lowercase
comment_df["text"] = comment_df["text"].str.lower()

#remove newline 
comment_df["text"] = comment_df["text"].str.replace("\n", " ")

In [47]:
comment_words = flatten(comment_df.text.str.split(" ").to_list())

# Removing stopwords and stemming

In [48]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
hu_stop = stopwords.words("hungarian")
print(hu_stop)

['a', 'ahogy', 'ahol', 'aki', 'akik', 'akkor', 'alatt', 'által', 'általában', 'amely', 'amelyek', 'amelyekben', 'amelyeket', 'amelyet', 'amelynek', 'ami', 'amit', 'amolyan', 'amíg', 'amikor', 'át', 'abban', 'ahhoz', 'annak', 'arra', 'arról', 'az', 'azok', 'azon', 'azt', 'azzal', 'azért', 'aztán', 'azután', 'azonban', 'bár', 'be', 'belül', 'benne', 'cikk', 'cikkek', 'cikkeket', 'csak', 'de', 'e', 'eddig', 'egész', 'egy', 'egyes', 'egyetlen', 'egyéb', 'egyik', 'egyre', 'ekkor', 'el', 'elég', 'ellen', 'elõ', 'elõször', 'elõtt', 'elsõ', 'én', 'éppen', 'ebben', 'ehhez', 'emilyen', 'ennek', 'erre', 'ez', 'ezt', 'ezek', 'ezen', 'ezzel', 'ezért', 'és', 'fel', 'felé', 'hanem', 'hiszen', 'hogy', 'hogyan', 'igen', 'így', 'illetve', 'ill.', 'ill', 'ilyen', 'ilyenkor', 'ison', 'ismét', 'itt', 'jó', 'jól', 'jobban', 'kell', 'kellett', 'keresztül', 'keressünk', 'ki', 'kívül', 'között', 'közül', 'legalább', 'lehet', 'lehetett', 'legyen', 'lenne', 'lenni', 'lesz', 'lett', 'maga', 'magát', 'majd', 'majd

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\balin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
with open(r"..\data\misc\hu_stopwords.txt", "r",  encoding='utf-8') as f:
    additional_stopwords = f.readlines()

additional_stopwords = [ word.replace("\n","") for word in additional_stopwords]
hu_stop += additional_stopwords
hu_stop = list(set(hu_stop))

In [62]:
#filtering stopwords
def filterStopword(word):
    if word in hu_stop:
        return False
    else:
        return True

filtered_text = list(filter(filterStopword, comment_words))

In [63]:
#remove '' blank str
filtered_text = list(filter(lambda a: a != "", filtered_text))

In [65]:
cntr = collections.Counter( filtered_text )
ws, cnt = zip( *cntr.most_common() )

fig = go.Figure([go.Bar(x=ws[:20], y=cnt[:20], marker_color = 'rgb(17,165,121)')])
fig.update_yaxes(type="log")

fig.update_traces(marker_line_color='rgb(17,165,121)',
                  marker_line_width=0.5, opacity=0.6)

fig.update_layout(title="Barchart of the most frequent words",
                  yaxis_title="word frequency",)

fig.show()

In [45]:
ws

('is',
 'ha',
 'mzp',
 'fidesz',
 '\n\n',
 'dobrev',
 '\n',
 'szerintem',
 'ő',
 'karácsony',
 'dk',
 'es',
 'se',
 'szavazni',
 'gyurcsány',
 'fog',
 'le',
 'orbán',
 'ellenzéki',
 'ember',
 'miatt',
 'tudom',
 'jelölt',
 'magyar',
 'inkább',
 't',
 'fideszes',
 'tényleg',
 'kb',
 'emberek',
 'első',
 'hát',
 'politikai',
 'amúgy',
 'biztos',
 'jobb',
 'the',
 'ellenzék',
 'annyira',
 'na',
 'online',
 'kicsit',
 'pont',
 'két',
 'második',
 'sajnos',
 'jakab',
 'teljesen',
 'ők',
 'párt',
 'miniszterelnök',
 'tudja',
 'te',
 'szóval',
 'megy',
 'ben',
 '\n\na',
 'mondjuk',
 'pár',
 'mszp',
 'kormány',
 'h',
 'őket',
 'pl',
 'előválasztás',
 'zay',
 'megint',
 'senki',
 'ország',
 'márki',
 'tud',
 'péter',
 'kéne',
 'd',
 'nyer',
 'év',
 'kellene',
 'nekik',
 'ban',
 'értem',
 'mind',
 'mar',
 'alapján',
 'túl',
 'kis',
 'ön',
 'előtt',
 're',
 'ma',
 'karigeri',
 'remélem',
 'gondolom',
 'akar',
 'esetleg',
 'rossz',
 'jobbik',
 'stb',
 'fogja',
 'fognak',
 'to',
 'dolog',
 'egyébké